In [1]:
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import time
import json
import pymysql

In [2]:
def read_config(p:str) -> dict:
    """
    p: config file 경로
    """
    with open(p, 'r') as f:
        lines = f.readlines()
    config_dict={}

    for l in lines:
        idx = l.index('=')
        k = l[:idx]
        v = l[idx+1:]
        config_dict[k] = v.rstrip()
    
    return config_dict

In [3]:
baseballdb_config = read_config('./baseballdb_config')
conn = pymysql.connect(**baseballdb_config)

In [4]:
def bb_pitcher():
    now = dt.datetime.now()
    ny = now.strftime('%Y')
    nd = now.strftime('%Y%m%d')
    nm = now.strftime('%m')
    x = '4'
    result_lst = []
    for month in range(int(x.zfill(2)), int(nm)+1):
        url=f'https://sports.news.naver.com/kbaseball/schedule/index?date={nd}&month={month}&year={ny}&teamCode='
        response = requests.get(url)
        soup = bs(response.text, 'lxml')
        divs = soup.select('#calendarWrap>div')
        for div in divs:
            trs = div.select('tr')
            for tr in trs:
                tds = tr.select('td')
                if tr == trs[0]:
                    if len(tds) == 3:
                        break
                    elif tr.select('td.add_state'): 
                        win_pit_name = ''
                        win_pit_inn = ''
                        win_pit_hit = ''
                        win_pit_bbhp = ''
                        win_pit_kk = ''
                        win_pit_lr = ''
                        win_pit_pn = ''
                        win_pit_era = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                    elif tr.find_all('img')[2]['alt'] == "경기결과":
                        if tr.select('strong.td_score')[0].text.split(':')[0] == tr.select('strong.td_score')[0].text.split(':')[1]:
                            win_pit_name = ''
                            win_pit_inn = ''
                            win_pit_hit = ''
                            win_pit_bbhp = ''
                            win_pit_kk = ''
                            win_pit_lr = ''
                            win_pit_pn = ''
                            win_pit_era = ''
                            win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                        else:
                            raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                            record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                            record_rp = requests.get(record_url)
                            record = json.loads(record_rp.text)
                            pit_record = record['result']['recordData']['pitchersBoxscore']
                            win_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or \
                                            [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']
                            for wp in win_pit_info:
                                win_pit_name = wp['name']
                                win_pit_inn = wp['inn']
                                win_pit_hit = str(wp['hit'])
                                win_pit_bbhp = str(wp['bbhp'])
                                win_pit_kk = str(wp['kk'])
                                win_pit_lr = str(wp['r'])+'('+str(wp['er'])+')'
                                win_pit_pn = str(wp['bf'])
                                win_pit_era = str(wp['era'])
                                win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                    else:
                        win_pit_name = ''
                        win_pit_inn = ''
                        win_pit_hit = ''
                        win_pit_bbhp = ''
                        win_pit_kk = ''
                        win_pit_lr = ''
                        win_pit_pn = ''
                        win_pit_era = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)         
                else:
                    if len(tds) == 3:
                        break
                    if tr.select('td.add_state'): 
                        win_pit_name = ''
                        win_pit_inn = ''
                        win_pit_hit = ''
                        win_pit_bbhp = ''
                        win_pit_kk = ''
                        win_pit_lr = ''
                        win_pit_pn = ''
                        win_pit_era = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                    elif tr.find_all('img')[2]['alt'] == "경기결과":
                        if tr.select('strong.td_score')[0].text.split(':')[0] == tr.select('strong.td_score')[0].text.split(':')[1]:
                            win_pit_name = ''
                            win_pit_inn = ''
                            win_pit_hit = ''
                            win_pit_bbhp = ''
                            win_pit_kk = ''
                            win_pit_lr = ''
                            win_pit_pn = ''
                            win_pit_era = ''
                            win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                        else:
                            raw_record_url = tr.select('a')[0]['href'].replace('game', 'games')
                            record_url = 'https://api-gw.sports.naver.com/schedule'+raw_record_url
                            record_rp = requests.get(record_url)
                            record = json.loads(record_rp.text)
                            pit_record = record['result']['recordData']['pitchersBoxscore']
                            win_pit_info = [pit_info for pit_info in list(pit_record.values())[0] if pit_info['wls'] == '승'] or \
                                            [pit_info for pit_info in list(pit_record.values())[1] if pit_info['wls'] == '승']
                            for wp in win_pit_info:
                                win_pit_name = wp['name']
                                win_pit_inn = wp['inn']
                                win_pit_hit = str(wp['hit'])
                                win_pit_bbhp = str(wp['bbhp'])
                                win_pit_kk = str(wp['kk'])
                                win_pit_lr = str(wp['r'])+'('+str(wp['er'])+')'
                                win_pit_pn = str(wp['bf'])
                                win_pit_era = str(wp['era'])
                                win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)
                    else:
                        win_pit_name = ''
                        win_pit_inn = ''
                        win_pit_hit = ''
                        win_pit_bbhp = ''
                        win_pit_kk = ''
                        win_pit_lr = ''
                        win_pit_pn = ''
                        win_pit_era = ''
                        win_pit = (win_pit_name, win_pit_inn, win_pit_hit, win_pit_bbhp, win_pit_kk, win_pit_lr, win_pit_pn, win_pit_era)                    
                result_lst.append(win_pit)
    with conn.cursor() as cursor:
        sql = "SELECT `GameID` FROM `TB_game`"
        cursor.execute(sql) 
        result=cursor.fetchall()
    ID_list=list(result)
    ID_list
    ordered_lst=[]
    for i in range(len(result_lst)):
        tem_lst=[ID_list[i]+result_lst[i]]
        ordered_lst.append(tem_lst)
    pit_lst = [x for sublist in ordered_lst for x in sublist]
    return pit_lst
bb_pitcher()

[(1, '문성현', '1', '0', '0', '0', '0(0)', '14', '0.00'),
 (2, '김광현', '5', '4', '3', '3', '1(1)', '87', '1.80'),
 (3, '벤자민', '6', '2', '0', '4', '1(0)', '87', '0.00'),
 (4, '최지강', '0 ⅔', '0', '0', '1', '0(0)', '9', '0.00'),
 (5, '페디', '5', '4', '3', '6', '0(0)', '108', '0.00'),
 (6, '김재웅', '1', '0', '0', '3', '0(0)', '12', '0.00'),
 (7, '이의리', '5', '3', '6', '3', '3(1)', '101', '1.80'),
 (8, '함덕주', '2', '0', '0', '4', '0(0)', '29', '0.00'),
 (9, '나균안', '6 ⅔', '5', '2', '4', '0(0)', '100', '0.00'),
 (10, '이승현', '2', '1', '0', '2', '0(0)', '26', '0.00'),
 (11, '이승현', '1', '1', '0', '0', '0(0)', '18', '0.00'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '플럿코', '5', '2', '4', '4', '0(0)', '89', '0.00'),
 (14, '오원석', '7', '2', '2', '6', '1(1)', '94', '1.29'),
 (15, '정철원', '1', '0', '1', '1', '0(0)', '13', '0.00'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '최원태', '6', '5', '3', '3', '1(1)', '104', '1.50'),
 (19, '', '', '', '', '', '', '', ''),
 (20, '

In [5]:
bbp=bb_pitcher()

In [6]:
bbp

[(1, '문성현', '1', '0', '0', '0', '0(0)', '14', '0.00'),
 (2, '김광현', '5', '4', '3', '3', '1(1)', '87', '1.80'),
 (3, '벤자민', '6', '2', '0', '4', '1(0)', '87', '0.00'),
 (4, '최지강', '0 ⅔', '0', '0', '1', '0(0)', '9', '0.00'),
 (5, '페디', '5', '4', '3', '6', '0(0)', '108', '0.00'),
 (6, '김재웅', '1', '0', '0', '3', '0(0)', '12', '0.00'),
 (7, '이의리', '5', '3', '6', '3', '3(1)', '101', '1.80'),
 (8, '함덕주', '2', '0', '0', '4', '0(0)', '29', '0.00'),
 (9, '나균안', '6 ⅔', '5', '2', '4', '0(0)', '100', '0.00'),
 (10, '이승현', '2', '1', '0', '2', '0(0)', '26', '0.00'),
 (11, '이승현', '1', '1', '0', '0', '0(0)', '18', '0.00'),
 (12, '', '', '', '', '', '', '', ''),
 (13, '플럿코', '5', '2', '4', '4', '0(0)', '89', '0.00'),
 (14, '오원석', '7', '2', '2', '6', '1(1)', '94', '1.29'),
 (15, '정철원', '1', '0', '1', '1', '0(0)', '13', '0.00'),
 (16, '', '', '', '', '', '', '', ''),
 (17, '', '', '', '', '', '', '', ''),
 (18, '최원태', '6', '5', '3', '3', '1(1)', '104', '1.50'),
 (19, '', '', '', '', '', '', '', ''),
 (20, '

In [8]:
with conn.cursor() as cursor:
    sql="INSERT INTO `TB_winpitcher`(`GameID`, `Name`, `Inning`, `Hit`, `BBHP`, `StrikeOut`, `Lose_Run`, `PitchingNumber`, `Era`) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, bbp)
conn.commit()